feature columns时raw data和Estimator的中介。Feature columns非常丰富，让你可以将大范围的raw data转换为Estimator可以用的格式。

在[pre-made Estimator](https://www.tensorflow.org/get_started/premade_estimators?hl=zh-cn)， 我们可以用premade Estimator，DNNClassifier来训练一个可以预测iris不同花的种类的模型。哪个示例用 数值feature columns( __tf.feature_column.numeric_column__ )。 但是现实中很多数据的类型不是数值类型的。
![Title](feature_cloud.jpg)
## Input to a Deep Neural Network
DNN可以操作数字（例如，__tf.float32__) 。总而言之，在NN当中的每个neuron都是在weights和输入数据上做乘法和假发运算。 在现实生活中的输入数据，经常会包含非数值（categorical)数据。例如，一个product_class 类型包含了以下三个非数值的内容：
* kitchenware
* electronics
* sports
ML模型经常用简单的向量来表达categorical值，1表示这个值存在，0表示不存在，所以经常用 [0, 0, 1] 表示product_class,表明：（即 one_hot encoding)
* 0: kitchenware is absent
* 0: electronics is absent
* 1: sports is present
所以虽然raw data当中由数值类型和分类类型，ML模型将这些特征都用数字表示。
## Feature Columns
在下面的图形中表示，你可以通过Estimator中的__feature_columns__参数来指定一个模型的输入。Feature Columns桥接模型当中的input data（通过input_fn。
![Title](inputs_to_model_bridge.jpg)
调用 __tf.feature_column__ 模块当中的函数可以创建feature columns。这个文档解释了这个模块当中的九个函数。在余下的图形中显示，这九个函数要么返回一个Categorical-Columns要么返回一个Dense-Column对象，除了__bucketized_column__, 继承于这两个类。
![Title](some_constructors.jpg)

### Numeric column
Iris classifier 对所有的输入特征调用 __tf.feature_column.numeric_column__ :
* SepalLength
* SepalWidth
* PetalLength
* PetalWidth
虽然 __tf.numeric_column__ 的参数都是可以选择的，可以调用 __tf.numeric_column__ 而不带任何其他的参数，如下1，是一个为模型指定默认数值类型(__tf.float32__)得数值值得方法。

如果要指定一个非数值数据类型，需要用到__dtype__参数，如下2：

默认情况下，一个数值column创造一个标量值，可以通过shape参数来定义另外一种shape，如下3：

In [2]:
import tensorflow as tf
# 1：
# Defaults to a tf.float32 scalar.
numeric_feature_column = tf.feature_column.numeric_column(key="SepalLength")

# 2:
# Represent a tf.float64 scalar.
numeric_feature_column = tf.feature_column.numeric_column(key="SepalLength",
                                                         dtype=tf.float64)

# 3:
# Represent a 10-element vector in which each cell contains a tf.float32.
vector_feature_column = tf.feature_column.numeric_column(key="Bowling",
                                                         shape=10)

# Represent a 10x5 matrix in which each cell contains a tf.float32.
matrix_feature_column = tf.feature_column.numeric_column(key="MyMatrix",
                                                         shape=[10,5])

### Bucketized column
通常情况下，你不会将一个数传递给模型，你会将他们的值基于数值区间分到不同的categories。要这样做，可以创造一个 [bucketized column](https://www.tensorflow.org/api_docs/python/tf/feature_column/bucketized_column?hl=zh-cn)。例如，考虑代表房子建造年份的原始数据(raw data)。我们不将年份看成是一个标量的numeric column，我们将年份分为以下的四个buckets。
![Title](bucketized_column.jpg)
这个模型代表的bucket如下：

Date Range|	Represented as...
 :--:     | :--: 
< 1960	|[1, 0, 0, 0]
>= 1960 but < 1980	|[0, 1, 0, 0]
>= 1980 but < 2000	|[0, 0, 1, 0]
> 2000	|[0, 0, 0, 1]

这么处理的原因：在这里，这个单一的输入的值变成了一个四维向量。因此，这个模型需要学习四个weights而不是一个，四个weights会比1 weight创造更加richer的模型。更加重要的时，bucketizing让模型能够更加明显的区分出year categories因为在四维向量中只有一个元素为1。当我们直用一个单一的数值（a year）作为输入时，模型只可以学习一个线性关系。所以，bucketing提供了可以让模型学习到更多的灵活性。  
以下code展示如何来创建一个 bucketized feature：

注意，制定一个三维的boundaries可以创建一个四维的 bucketized vector.

In [ ]:
# First, convert the raw input to a numeric column.
numeric_feature_column = tf.feature_column.numeric_column("Year")

# Then, bucketize the numeric column on the years 1960,1980, and 2000.
bucketized_feature_column = tf.feature_column.bucketized_column(
    source_column = numeric_feature_column,
    boundaries = [1960, 1980, 2000]
)

### Categorical identity column(分类标识列)
分类标识列可以被荣威时bucketized columns的特别情况，在典型的bucketized columns，每一个bucket都代表一个范围的数值。 在分裂标识列，每一个bucket代表一个单一唯一的证书。例如，如果想表达整数的范围\[0,4)，也就是说表达整数0，1，2，3，在这个情形下，分类标识列的映射类似于下图：
![Title](categorical_column_with_identity.jpg)
与bucketized columns一样，一个model可以为分类标识列的每一个类学习一个不同的权重。例如，如果不用一个字符串来表示 __product_class__ ,而用单独的整数数值表示，也就是：
* 0="kitchenware"
* 1="electronics"
* 2="sport"

调用 _tf.feature_column.categorical_column_with_identity 来实现分类标识列。例如：

In [9]:
# Create categorical output for an integer feature named "my_feature_b",
# The values of my_feature_b must be >= 0 and < num_buckets
identity_feature_column = tf.feature_column.categorical_column_with_identity(
    key = "my_feature_b",
    num_buckets = 4 # Values=[0,4]
)
# In order for the preceding call to work, the input_fn() must return
# a dictionary containing 'my_feature_b' as a key. Furthermore, the values
# assigned to 'my_feature_b' must belong to the set [0, 4).
def input_fn():
    ...
    return({'my_feature_a':[7, 9, 5, 2], 'my_feature_b':[3, 1, 2, 2] },
            [Label_values])

### Categorical vocabulary column（词汇分类列）
一个输入字符串时无法直接传递给一个模型的，必须要将字符串映射到numeric或者categorical value。categorical vocabulary columns提供一个把字符串表达成one-hot vector的方法。如图所示：
![Titile](categorical_column_with_vocabulary.jpg)
如图所示，词汇分类列时一种分类标识列的枚举类型。TensorFlow提供两种不同的函数来创建词汇分类列：
* __tf.feature_column.categorical_column_with_vocabulary_list__
* __tf.feature_column.categorical_column_with_vocabulary_file__
__categorical_column_with_vocabulary_list__ 将字符串基于一个显示的字典列表映射成整数，如下例1：

这个函数时非常直观的，但是有一个很大的缺点。比如说，当字典很长的时候，需要大很多的子。在这种情况下，  __tf.feature_column.categorical_column_with_vocabulary_file__ 可以用独立的file代替词汇表。如下例2：

In [10]:
# 1:
# Given input "feature_name_from_input_fn" which is a string,
# create a categorical feature by mapping the input to one of
# the elements in the vocabulary list
vocabulary_feature_column= tf.feature_column.categorical_column_with_vocabulary_list(
    key = "a feature returned by input_fn()",
    vocabulary_list = ["kitchenware","electronics","sports"]
)

# 2:
# Given input "feature_name_from_input_fn" which is a string,
# create a categorical feature to our model by mapping the input to one of
# the elements in the vocabulary file
vocabulary_feature_column = tf.feature_column.categorical_column_with_vocabulary_file(
    key = "a feature returned by input_fn()",
    vocabulary_file = "product_class.txt",
    vocabulary_size=3
)
## 在product_class.txt中需要包含所有词汇元素，而且每行即是一个词汇元素，
## 在本例当中，内容为
## kitchenware\n
## electronics\n
## sports

### Hashed Column
至今为止，我们只接触了少数几个类别。例如，product_class 示例当中只有3个categories。一般情况下，类别的数量很大，以至于无法将每个词汇或者整数拥有一个单独的分类，因为这样的话会消耗台大的内存。在这些情况下，我们需要问一个问题——在输入中有多少个类是所需的。 事实上， __tf.feature_column.categorical_column_with_hash_bucket__ 函数保证你可以指定类别的数量。在这种feature column的类型当中，model计算输入的hash值，然后通过模运算（modulo operator)将他们放入 __hash_bucket_size__ 分类当中，伪代码如下1所示。

这个代码中创建的 __feature_column__  如下2所示的代码。

在这个时候，你可能回想，这很疯狂。毕竟，我们强制让不同的输入值最后只划分为少数的类别。这意味着两个可能不相关的输入可能会划分到同一个类别当中，这样神经网络就会无差别对待他们。下面的图片表明这个困境，展示了ketcheware和sports都被划分到了同一个类别(hash bucket) 12:
![Title](hashed_column.jpg)
跟在机器学习当中其他的违反直觉的情况相同，hashing在实际情况中经常运行很好(works well)。这是因为hash分类为模型提供了一些分离(separation),模型可以运用一些额外的特征来将kitchenware和sports分开。

In [14]:
# 1：
#pseudocode
feature_id = hash(raw_feature) % hash_buckets_size

# 2:
hashed_feature_column = \
tf.feature_column.categorical_column_with_hash_bucket(
    key = "some_feature",
    hash_bucket_size = 100 # The number of categories
)

### Crossed Column(特征交叉）
将一些特征组合到一个特征，就是feature cross,让模型可以专门针对特征组合表示的任何意义学习单独的权重。

更具提的，假设我们想要模型可以计算在Atalanta.GA 计算房地产(estate)的实际价格。在这个城市，房地产的价格与地理位置有很大的关系，把精度和纬度表达成不同的特征不能够在确定房地产的位置相关性上面很有用；但是，将经度和纬度组合成一个特征可以确定位置。假设我们将Atlanta表示成一个100x100的矩形网格图，并且依照由经度和纬度组成的特征分割为10，000块。这个特征交叉可以让这个模型学习不同的分割快与定价的关系，要比单独的利用纬度和经度强得多。

下面就是Atlanta的平面图，在红色的文字部分显示着这个城市四个角的纬度和经度的值：
![Title](Atlanta.jpg)
在这个解决方案当中，使用了 __bucketized_column__ 和 __tf.feature_column.crossed_column__ 函数：

In [ ]:
def make_dataset(latitude,longitude,labels):
    assert latitude.shape == longitude.shape ==labels.shape
    
    features = {'latitude':latitude.flatten(),
                'longitude':longitude.flatten()}
    labels=labels.flatten()
    
    return tf.data.Dataset.from_tensor_slices((features,labels))

# Bucketize the latitude and longitude using the 'edges'
latitude_bucket_fc = tf.feature_column.bucketized_column(
    tf.feature_column.numeric_column('latitude'),
    list(atlanta.latitude.edges)
)

longitude_bucket_fc = tf.feature_column.bucketized_column(
    tf.feature_column.numeric_column('longitude'),
    list(atlanta.longitude.edges)
)

# Cross the bucketized columns, using 5000 hash bins.
crossed_lat_lon_fc = tf.feature_column.crossed_column(
    [latitude_bucket_fc,longitude_bucket_fc],5000
)

fc = [
    latitude_bucket_fc,
    longitude_bucket_fc,
    crossed_lat_lon_fc]

# Build and train the Estimator.
est = tf.estimator.LinearRegressor(fc, ...)

可以通过以下任意信息做特征交叉：
* Feature names（特征名称），也就是说从input_fn中返回的dict的名字。
* 任意类别列，除了 categorical_column_with_hash_bucket（因为 crossed_column 散列了输入(hashes the input)）

当特征列__latitude_bucket_fc__和__longitude_bucket_fc__交叉，TensorFlow会为每个例子创建(latitude_fc,longitude_fc)对，产生所有可能的组：

 (0,0),  (0,1)...  (0,99)
 (1,0),  (1,1)...  (1,99)
   ...     ...       ...
(99,0), (99,1)...(99, 99)

只有有限的字典才能够产生可跟踪的全部的网格。如果不需要对很大的输入表产生全部的组合，那么 __crossed_column__ 可以通过参数 __hash_bucket_size__ 的设置产生相应数目的组合。这些特征列通过在一个元组的输入上运行一个hash函数，然后再对其进行模运算（模 __hash_bucket_size__ )来指定一个example到一个索引(index)。

与之前讨论一致， 运行hash和模运算会限制列别的数目，会造成类别碰撞(category collisions），也就是说，多个(latitude,longitude)特征交叉会最后再同一个hash bucket当中。但是再实际情况中，特征交叉仍然对模型的学习能力上有很大的提升。

虽然有一些反直觉的，当创建特征交叉的时候，你实际上在仍然应到包括模型的原始特征，如示例所示。不相关的纬度和经度的特征帮助模型在交叉特征出现hash collision的时候能够帮助模型来区分 examples。

## Indicator and embedding columns（指示列和嵌入列）
### Indicator Columns
指示列和嵌入列从来不直接作用在特征上面，而是将类别列(categorical columns）作为输入。

当使用指示列的时候，我们告诉TensorFlow做我们在categorical product_class 示例当中看到的。也就是说，一个指示列将每个列别作为 one-hot vector当中的一个元素，而匹配
类别值是1，其他的都是0，如下图所示：
![Title](categorical_column_with_identity.jpg)
这里，你可以通过调用 __tf.feature_column.indicator_column__ 创建指示列。

In [ ]:
categorical_column = ... # Create any type of categorical column.

# Represent the categorical column as an indicator column.
indicator_column = tf.feature_column.indicator_column(categorical_column)

### Embedding Columns
假设，我们不是只有三个可能的类别，而有一百万个，甚至十亿个。处于一些原因，就像类别的数目增长的很大的情况，这样就不可能训练一个神经网络使用指示列。

我们可以利用嵌入列来战胜这个局限性。不是用多维的one_hot vector来表示数据，而用embedding column 表示一个每个单元都可以包含任意数值的，低维的，普通的vector来表示数据。通过允许每个单元可以有任何的数字，获得更多得数字组合，嵌入列的维度比指示列少得多。

通过一个示例来比较指示列和嵌入列。假设我们得输入例子当中包含一个只有81个词得有限组合。假设这个数据集合提供以下得4个不同得输入词：
* "dog"
* "spoon"
* "scissors"
* "guitar"
在这个例子中，下面这张图代表了嵌入列和指示列不同的处理路径：
![Title](embedding_vs_indicator.jpg)
一个嵌入列用比嵌入列低维的矢量来存储分类数据（可以将任意数据填入嵌入列当中，训练会确定他们真正的值）  

当一个例子运行的时候，一个 __categorical_colum_with...__ 函数将示例字符串映射到一个数字分类值(numerical categorical value)。例如，方程将"spoon"映射到\[32]\(32来自于我们的想象——针式的值依赖于映射函数)。你可以通过下面两种方法来表达这些数字分类值：
* 作为indicator column，一个方程将每个数字分类值转换到81元素的矢量(81-element vector)上（因为在组合(platte)上包含81个词汇）,在分类值(0, 32, 79, 80) 的索引上放置1，其他地方放0.
* 作为嵌入列。 函数利用数字分类值(0, 32, 79, 80) 作为索引来查阅表，在查阅表上每一个位置都包含了一个三维的矢量(3-element vector)。

在embedding vector里的数值怎么样能够被分配呢？事实上，这个值得分配发生在训练得过程当中。也就是说，模型学习一个最好得方式来将输入得数字分类值映射到对应得embeddings vector值当中来解决这个问题。 嵌套列因为每一个embeddings vector可以学习训练数据中得新得分类关系从而增加model得能力。

为什么在示例当中，embedding vector得维度是3呢？ 下面得方程当中包含了一个关于潜入维度的一般的经验法则：  
__embedding_dimensions = number_of_categories**0.25

也就是说，嵌入矢量的维度应该是类别数目的1/4次方。因为在这个示例中的词汇大小是81，所以建议的维度是3。

注意，这只是一个一般的标准，你可以将嵌入维度设置为任意数目大小。

调用 __tf.feature_column.embedding_column__ 可以创造一个 __embedding_column__ , 参照以下代码示例：

embbeding 在机器学习当中是一个重要的话题，更加详细的参见embedding 单元

In [ ]:
categorical_column = ... # Create any categorical column

# Represent the categorical column as an embedding column.
# This means creating a one-hot vector with one element for each category.
embedding_column = tf.feature_column.embedding_column(
    categorical_colum=categorical_column,
    dimension=dimension_of_embedding_vector
)

## Passing feature columns to Estimators
在下面的列表中显示，不是所有的Estimators都允许所有的 feature_columns 参数：
* __LinearClassifier__ 和 __LinearRegressor：__ 接受所有的feature columns的类别。
* __DNNClassifier__ 和 __DNNRegressor:__ 只接受dense columns.其他的column types需要被包装(wrapped) __indicator_column__ 或者 __embedding_column__ 中。
* __DNNLinearCombinedClassifier__ 和 __DNNLinearCombinedRegressor：__  
    * __linear_feature_columns__ 参数接受任意特征列类型。
    * __dnn_feature_columns__ 参数只接受dense columns

## Other Source
* [Low Level Introduction](https://www.tensorflow.org/programmers_guide/low_level_intro?hl=zh-cn#feature_columns)示例怎么样用在低级别的API上使用 feature_columns
* [wide](https://www.tensorflow.org/tutorials/wide?hl=zh-cn)和[Wide&Deep](https://www.tensorflow.org/tutorials/wide_and_deep?hl=zh-cn)解决在二分类问题当中怎么样在大部分的输入数据类型中使用feature_columns。

要学习更多的embedding的只是，查看如下：
(http://colah.github.io/posts/2014-07-NLP-RNNs-Representations/)